# Q1. Baseline Probability Calculation

In [2]:
import numpy as np

# Define the transition matrix
P = np.array([
    [0.99, 0.01, 0.0],
    [0.0, 0.99, 0.01],
    [0.0, 0.0, 1.0]
])

# Initial state vector: starting from Intact
pi_0 = np.array([1.0, 0.0, 0.0])

# Function to compute state probabilities after t steps
def compute_state_prob(pi_0, P, t):
    P_t = np.linalg.matrix_power(P, t)
    return pi_0 @ P_t

# Compute state probabilities after 50 and 100 weeks
pi_50 = compute_state_prob(pi_0, P, 50)
pi_100 = compute_state_prob(pi_0, P, 100)

# Print the results
print(f"t = 50: Intact={pi_50[0]:.4f}, Damage={pi_50[1]:.4f}, Failure={pi_50[2]:.4f}")
print(f"t = 100: Intact={pi_100[0]:.4f}, Damage={pi_100[1]:.4f}, Failure={pi_100[2]:.4f}")

t = 50: Intact=0.6050, Damage=0.3056, Failure=0.0894
t = 100: Intact=0.3660, Damage=0.3697, Failure=0.2642


# Q2. Posterior Probability with Observations

In [6]:
import pandas as pd
import numpy as np

# Load observations from Measures.csv
df = pd.read_csv("Measures.csv", header=None)
measurements = df[0].values  # y_1 to y_50

# Define states and HMM parameters
states = [1, 2, 3]
lambda_values = {1: 0.5, 2: 0.75, 3: 1.0}  # Log-mean per state
zeta = 0.8  # Shared standard deviation (log-scale)

# Transition matrix
P = np.array([
    [0.99, 0.01, 0.0],
    [0.0, 0.99, 0.01],
    [0.0, 0.0, 1.0]
])

# Initial belief: fully Intact
pi_0 = np.array([1.0, 0.0, 0.0])

# Lognormal emission probability
def emission_prob(y, s):
    mu = lambda_values[s]
    sigma = zeta
    return (1 / (y * sigma * np.sqrt(2 * np.pi))) * np.exp(-((np.log(y) - mu) ** 2) / (2 * sigma ** 2))

# Forward algorithm for filtering
T = len(measurements)
n_states = len(states)
alpha = np.zeros((T, n_states))

# Initialization
for j in range(n_states):
    alpha[0, j] = pi_0[j] * emission_prob(measurements[0], states[j])
alpha[0] /= np.sum(alpha[0])  # Normalize

# Forward recursion
for t in range(1, T):
    for j in range(n_states):
        alpha[t, j] = emission_prob(measurements[t], states[j]) * np.sum(alpha[t-1] * P[:, j])
    alpha[t] /= np.sum(alpha[t])  # Normalize

# Filtered belief at t=50
pi_50 = alpha[-1]

# Predict forward 50 more steps (t=100)
pi_100 = pi_50 @ np.linalg.matrix_power(P, 50)

# Output
print(f"Posterior at t=50: Intact={pi_50[0]:.4f}, Damage={pi_50[1]:.4f}, Failure={pi_50[2]:.4f}")
print(f"Predicted at t=100: Intact={pi_100[0]:.4f}, Damage={pi_100[1]:.4f}, Failure={pi_100[2]:.4f}")

Posterior at t=50: Intact=0.5744, Damage=0.3883, Failure=0.0373
Predicted at t=100: Intact=0.3475, Damage=0.4104, Failure=0.2421


# Q3. Effect of Changed Emission Parameters

In [7]:
import pandas as pd
import numpy as np

# Load the same 50-week observations
df = pd.read_csv("Measures.csv", header=None)
measurements = df[0].values

# Set all lambda values equal → states are indistinguishable by observation
lambda_values = {1: 1.0, 2: 1.0, 3: 1.0}
zeta = 0.8  # same standard deviation

# Define transition matrix (same as before)
P = np.array([
    [0.99, 0.01, 0.0],
    [0.0, 0.99, 0.01],
    [0.0, 0.0, 1.0]
])

# Initial belief: fully Intact
pi_0 = np.array([1.0, 0.0, 0.0])

# Emission probability (lognormal) — same for all states
def emission_prob(y, s):
    mu = lambda_values[s]
    sigma = zeta
    return (1 / (y * sigma * np.sqrt(2 * np.pi))) * np.exp(-((np.log(y) - mu) ** 2) / (2 * sigma ** 2))

# Forward algorithm
T = len(measurements)
n_states = 3
alpha = np.zeros((T, n_states))

# Initialization
for j in range(n_states):
    alpha[0, j] = pi_0[j] * emission_prob(measurements[0], j+1)
alpha[0] /= np.sum(alpha[0])

# Filtering recursion
for t in range(1, T):
    for j in range(n_states):
        alpha[t, j] = emission_prob(measurements[t], j+1) * np.sum(alpha[t-1] * P[:, j])
    alpha[t] /= np.sum(alpha[t])  # Normalize

# Posterior at t=50
pi_50 = alpha[-1]

# Predict to t=100 using transition matrix
pi_100 = pi_50 @ np.linalg.matrix_power(P, 50)

# Output
print(f"Posterior at t=50: Intact={pi_50[0]:.4f}, Damage={pi_50[1]:.4f}, Failure={pi_50[2]:.4f}")
print(f"Predicted at t=100: Intact={pi_100[0]:.4f}, Damage={pi_100[1]:.4f}, Failure={pi_100[2]:.4f}")

Posterior at t=50: Intact=0.6111, Damage=0.3025, Failure=0.0864
Predicted at t=100: Intact=0.3697, Damage=0.3697, Failure=0.2605
